# Importing Libraries

In [1]:
import numpy as np
import json
import gzip
from scipy.sparse import coo_matrix
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Reimplementation of DEHNN Model

In [33]:
# Single layer of DEHNN model
class DEHNNLayer(nn.Module):
    def __init__(self, node_in_features, edge_in_features):
        super(DEHNNLayer, self).__init__()
        # Transformation for aggregating edge features to update node features
        self.node_mlp1 = nn.Sequential(nn.Linear(edge_in_features, edge_in_features), nn.ReLU())
        # Transformation for aggregating node features to update sink nodes
        self.edge_mlp2 = nn.Sequential(nn.Linear(node_in_features, node_in_features),nn.ReLU())
        self.edge_mlp3 = nn.Sequential(nn.Linear(2 * node_in_features, 2 * node_in_features),nn.ReLU())

        # Transformation for mapping node features to virtual nodes
        self.node_to_virtual_mlp = nn.Sequential(nn.Linear(node_in_features, node_in_features),nn.ReLU())
        # Transformation for updating higher virtual node features
        self.virtual_to_higher_virtual_mlp = nn.Sequential(nn.Linear(node_in_features, edge_in_features),nn.ReLU())
        # Transformation for updating virtual node features
        self.higher_virtual_to_virtual_mlp = nn.Sequential(nn.Linear(edge_in_features, edge_in_features),nn.ReLU())
        # Transformation for propagating virtual node features back to the original nodes
        self.virtual_to_node_mlp = nn.Sequential(nn.Linear(edge_in_features, edge_in_features),nn.ReLU())

        self.default_driver = nn.Parameter(torch.zeros(node_in_features))
        self.default_sink_agg = nn.Parameter(torch.zeros(node_in_features))
        self.default_edge_agg = nn.Parameter(torch.zeros(edge_in_features))
        self.default_virtual_node = nn.Parameter(torch.zeros(node_in_features))
        
        # Initialize weights of the network layers
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, node_features, edge_features, hypergraph):
        # Update node features by aggregating incident edge features
        updated_node_features = {}
        for node in hypergraph.nodes:
            incident_edges = hypergraph.get_incident_edges(node) # Get driver and sink nodes for each edge
            if incident_edges:
                # Aggregate features of edges incident to the node
                agg_features = torch.sum(torch.stack([self.node_mlp1(edge_features[edge]) for edge in incident_edges]), dim=0)
            else:
                agg_features = self.default_edge_agg
            updated_node_features[node] = agg_features

        # Updating the edge features by aggregating node features
        updated_edge_features = {}
        for edge in hypergraph.edges:
            driver, sinks = hypergraph.get_driver_and_sinks(edge)

            driver_feature = node_features[driver] if driver is not None else self.default_driver # Feature for driver node

            if sinks:
                # Aggregate features of sink nodes using edge_mlp2
                sink_agg = torch.sum(torch.stack([self.edge_mlp2(node_features[sink]) for sink in sinks]), dim=0)
            else:
                sink_agg = self.default_sink_agg
            # Concatenate driver and sink features, then update edge features using edge_mlp3
            concatenated = torch.cat([driver_feature, sink_agg])
            updated_edge_features[edge] = self.edge_mlp3(concatenated)

        # Aggregates the virtual nodes
        virtual_node_agg = {}
        for virtual_node in range(hypergraph.num_virtual_nodes):
            assigned_nodes = [node for node in hypergraph.nodes if hypergraph.get_virtual_node(node) == virtual_node]
            if assigned_nodes:
                # Aggregate features of nodes assigned to this virtual node using node_to_virtual_mlp
                agg_features = torch.sum(torch.stack([self.node_to_virtual_mlp(node_features[node]) for node in assigned_nodes]), dim=0)
            else:
                agg_features = self.default_virtual_node
            virtual_node_agg[virtual_node] = agg_features

        # Aggregate all virtual node features to form higher virtual node feature
        higher_virtual_feature = torch.sum(
            torch.stack([self.virtual_to_higher_virtual_mlp(virtual_node_agg[vn]) for vn in virtual_node_agg]), dim=0
        )
        # Propagate higher virtual node features to each virtual node
        propagated_virtual_node_features = {}
        for virtual_node in range(hypergraph.num_virtual_nodes):
            propagated_virtual_node_features[virtual_node] = self.higher_virtual_to_virtual_mlp(higher_virtual_feature)
        
        # Update node features by adding propagated virtual node features
        for node in hypergraph.nodes:
            virtual_node = hypergraph.get_virtual_node(node) # Get the virtual node for each node
            propagated_feature = self.virtual_to_node_mlp(propagated_virtual_node_features[virtual_node])
            updated_node_features[node] += propagated_feature # Add propagated feature to the node's feature

        return updated_node_features, updated_edge_features

# DEHNN class reimplementation
class DEHNN(nn.Module):
    def __init__(self, num_layers, node_in_features, edge_in_features):
        super(DEHNN, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        
        # Build the layers of the network
        for i in range(num_layers):
            self.layers.append(DEHNNLayer(node_in_features, edge_in_features)) # Add DEHNNLayer to the model
            node_in_features, edge_in_features = edge_in_features, node_in_features
            edge_in_features *= 2

        edge_in_features = edge_in_features // 2 # Restore the edge feature dimension
        self.output_layer = nn.Sequential(nn.Linear(node_in_features, 2)) # Final output layer for predictions

    def forward(self, node_features, edge_features, hypergraph):
        # Pass through all layers of the network
        for layer in self.layers:
            node_features, edge_features = layer(node_features, edge_features, hypergraph)
            
        # Stack the final node features and pass them through the output layer to get predictions
        final_node_features = torch.stack([node_features[node] for node in hypergraph.nodes], dim=0)
        output = self.output_layer(final_node_features)
        return output

# Basic Hypergraph
class Hypergraph:
    def __init__(self, nodes, edges, driver_sink_map, node_to_virtual_map, num_virtual_nodes):
        self.nodes = nodes
        self.edges = edges
        self.driver_sink_map = driver_sink_map
        self.node_to_virtual_map = node_to_virtual_map
        self.num_virtual_nodes = num_virtual_nodes

    # Method to get incident edges for a given node
    def get_incident_edges(self, node):
        return [edge for edge in self.edges if node in self.driver_sink_map[edge][1] or node == self.driver_sink_map[edge][0]]
    
    # Method to get the driver and sink nodes for a given edge
    def get_driver_and_sinks(self, edge):
        return self.driver_sink_map[edge]
    # Method to get the virtual node assigned to a given node
    def get_virtual_node(self, node):
        return self.node_to_virtual_map[node]

# Training Model with Preprocessed Xbar Data

In [ ]:
file_indices = range(2, 9)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Training the model with the designs 2-8 with epoch of 10 (due to computing reasons)
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(epochs):
    epoch_loss = 0  # Sum loss over all datasets for each epoch
    
    for i in file_indices:
        # Load data for the current file
        clean_data_dir = 'clean_data/'
        
        with open(f'{clean_data_dir}{i}.driver_sink_map.pkl', 'rb') as f:
            driver_sink_map = pickle.load(f)
        
        with open(f'{clean_data_dir}{i}.node_features.pkl', 'rb') as f:
            node_features = pickle.load(f)
        
        with open(f'{clean_data_dir}{i}.net_features.pkl', 'rb') as f:
            edge_features = pickle.load(f)
        
        with open(f'{clean_data_dir}{i}.congestion.pkl', 'rb') as f:
            congestion = pickle.load(f)
        
        partition = np.load(f'{clean_data_dir}{i}.partition.npy')
        
        # Turn data into nodes and edges
        node_features = {k: torch.tensor(v).float().to(device) for k, v in node_features.items()}
        edge_features = {k: torch.tensor(v).float().to(device) for k, v in edge_features.items()}
        
        nodes = list(range(len(node_features)))
        edges = list(range(len(edge_features)))
        hypergraph = Hypergraph(nodes, edges, driver_sink_map, partition, 2)
        
        # Recompute class weights for the current design to try to handle class imbalance
        num_congested = sum(value == 1 for value in congestion.values())
        num_not_congested = sum(value == 0 for value in congestion.values())
        total = num_congested + num_not_congested
        
        # Emphasizing congested class
        weight_for_class_0 = total / (2 * num_not_congested) * 0.5 if num_not_congested > 0 else 1.0
        weight_for_class_1 = total / (2 * num_congested) * 1.5 if num_congested > 0 else 1.0
        class_weights = torch.tensor([weight_for_class_0, weight_for_class_1]).to(device)
        
        # Update criterion for the current design
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        
        # Forward pass
        output = model(node_features, edge_features, hypergraph)
        
        # Target labels (binary: 0 for not congested, 1 for congested)
        target = torch.tensor(list(congestion.values())).to(device)

        loss = criterion(output, target)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()  # Reset gradients after each batch

        epoch_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/10], Loss: {epoch_loss:.4f}")

Epoch [1/10], Loss: 16.1228
Epoch [2/10], Loss: 8.4986
Epoch [3/10], Loss: 9.9067
Epoch [4/10], Loss: 6.2397
Epoch [5/10], Loss: 7.6678
Epoch [6/10], Loss: 5.9226
Epoch [7/10], Loss: 6.7864
Epoch [8/10], Loss: 5.9236
Epoch [9/10], Loss: 6.3125
Epoch [10/10], Loss: 6.5801


# Testing Model on Design 13 from XBar

In [79]:
# Testing the data on design 13

clean_data_dir = 'clean_data/'
i = 13
with open(f'{clean_data_dir}{i}.driver_sink_map.pkl', 'rb') as f:
    driver_sink_map = pickle.load(f)

with open(f'{clean_data_dir}{i}.node_features.pkl', 'rb') as f:
    node_features = pickle.load(f)

with open(f'{clean_data_dir}{i}.net_features.pkl', 'rb') as f:
    edge_features = pickle.load(f)

with open(f'{clean_data_dir}{i}.congestion.pkl', 'rb') as f:
    congestion = pickle.load(f)

partition = np.load(f'{clean_data_dir}{i}.partition.npy')

# Turn data into nodes and edges
node_features = {k: torch.tensor(v).float().to(device) for k, v in node_features.items()}
edge_features = {k: torch.tensor(v).float().to(device) for k, v in edge_features.items()}

test_output = model(node_features, edge_features, Hypergraph(node_features, edge_features, driver_sink_map, partition, 2))

out = test_output.detach().cpu().numpy()
out = np.array([np.argmax(i) for i in out])

In [80]:
# Actual labels
true_labels = np.array(list(congestion.values()))
# Predicted labels
predicted_labels = out
# Classification Report
class_report = classification_report(true_labels, predicted_labels)
print('Accuracy: ' + str(np.mean(np.array(list(congestion.values())) == out)))
print("\nClassification Report:\n", class_report)

Accuracy: 0.7364080635308491

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.91      0.84      5117
           1       0.25      0.11      0.15      1431

    accuracy                           0.74      6548
   macro avg       0.52      0.51      0.50      6548
weighted avg       0.67      0.74      0.69      6548



In [70]:
model.layers # Viewing each layer in the trained model

ModuleList(
  (0): DEHNNLayer(
    (node_mlp1): Sequential(
      (0): Linear(in_features=1, out_features=1, bias=True)
      (1): ReLU()
    )
    (edge_mlp2): Sequential(
      (0): Linear(in_features=14, out_features=14, bias=True)
      (1): ReLU()
    )
    (edge_mlp3): Sequential(
      (0): Linear(in_features=28, out_features=28, bias=True)
      (1): ReLU()
    )
    (node_to_virtual_mlp): Sequential(
      (0): Linear(in_features=14, out_features=14, bias=True)
      (1): ReLU()
    )
    (virtual_to_higher_virtual_mlp): Sequential(
      (0): Linear(in_features=14, out_features=1, bias=True)
      (1): ReLU()
    )
    (higher_virtual_to_virtual_mlp): Sequential(
      (0): Linear(in_features=1, out_features=1, bias=True)
      (1): ReLU()
    )
    (virtual_to_node_mlp): Sequential(
      (0): Linear(in_features=1, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (1): DEHNNLayer(
    (node_mlp1): Sequential(
      (0): Linear(in_features=28, out_features=28, bias=Tru